## **1 кейс**

**Работа с логами**

**Важно**

Перед началом решения выполните следующую ячейку, чтобы загрузить необходимый для работы файл.

In [1]:
!wget https://gist.github.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log

--2025-03-24 06:58:16--  https://gist.github.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log
Resolving gist.github.com (gist.github.com)... 20.27.177.113
Connecting to gist.github.com (gist.github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log [following]
--2025-03-24 06:58:17--  https://gist.githubusercontent.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 459418 (449K) [text/plain]
Saving to: ‘auto_purchase.log’

auto_purchase.log   100%[===================>] 448.65K  2.24MB/s    in 0.2s    

2025-03-24 06:58:18 (2.

Чтобы посмотреть как он выглядит выполните следующую ячейку.

In [ ]:
with open('auto_purchase.log', 'r') as f:
    lines = f.readlines()

for line in lines[:15]:
    print(line)

INFO  | 2022-07-19 16:46:43,708  | file: demon_auto_purchase_bundle.py |  line: 44 | [demon] Поверяем истекающие премиум-пакеты

INFO  | 2022-07-19 16:46:43,722  | file: demon_auto_purchase_bundle.py |  line: 54 | [demon] Cегодня 2022-07-19, количество людей с автопродлением подписки: 1

INFO  | 2022-07-19 16:46:43,723  | file: demon_auto_purchase_bundle.py |  line: 61 | [demon] Обновляем подписку пользователю id: 63, payment_method_id: 2a68f0fe

INFO  | 2022-07-19 17:05:32,925  | file: demon_auto_purchase_bundle.py |  line: 44 | [demon] Поверяем истекающие премиум-пакеты

INFO  | 2022-07-19 17:05:32,944  | file: demon_auto_purchase_bundle.py |  line: 54 | [demon] Cегодня 2022-07-19, количество людей с автопродлением подписки: 1

INFO  | 2022-07-19 17:05:32,947  | file: demon_auto_purchase_bundle.py |  line: 61 | [demon] Обновляем подписку пользователю id: 63, payment_method_id: 2a68f0fe

INFO  | 2022-07-20 00:01:02,711  | file: demon_auto_purchase_bundle.py |  line: 44 | [demon] Повер

### **Решения**

#### **Задача 1**

Ваша задача написать функцию `count_success_and_failure`, которая принимает на вход путь к файлу с логами и подсчитывает количество успешных продлений и ошибок при списании. Функция должна вернуть кортеж из двух значений: количества успешных попыток и неуспешных.

**Решение**

Напишите свое решение ниже

In [ ]:
def count_success_and_failure(file_path):

    with open(file_path, 'r') as file:
      reader = file.readlines()
      return len([
          row for i, row in enumerate(reader) if (i < len(reader) - 1 and 'Обновляем подписку' in row and not reader[i+1].startswith('ERROR')) or
           ('Обновляем подписку' in row and i >= len(reader) - 1)
          ]), len([
              row for row in reader if row.startswith('ERROR')])

count_success_and_failure('auto_purchase.log')

(1034, 186)

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [ ]:
res = count_success_and_failure('auto_purchase.log')

try:
    assert res == (1034, 186)
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


#### **Задача 2**

Ваша задача написать функцию `auto_renewal_sub`, которая принимает на вход путь к файлу с логами и обрабатывает количество клиентов с автопродлением подписки. Мы хотим посмотреть на изменение этого показателя в динамике: посчитайте сглаженные значения с помощью метода скользящего среднего и метода медианного сглаживания.  

**Примечание:** При сглаживании берем все предыдущие значения, включая текущее, будущие значения не берем. Если в один день наблюдаем несколько записей об автопродлении - берем максимальное из имеющихся число клиентов с подпиской.

Функция должна записать в файл `auto_renewal_sub.txt` два списка, предварив их соответствущими обозначениями:

`Среднее: [2.0, 1.0, 0.67...]`

`Медиана: [2, 2, 0...]`

**Решение**

Напишите свое решение ниже

In [2]:
from collections import defaultdict

In [ ]:
#функции отбора уникального значения (максимальное),
#расчета среднего значения и медианы
def get_max(cnt_users):
  grouped = defaultdict(list)
  for key, value in cnt_users:
    grouped[key].append(value)
  return [max(values) for values in grouped.values()]

def get_avg(cnt_users):
  for i in range(len(cnt_users)):
      avg = sum(cnt_users[:i+1]) / (len(cnt_users[:i]) + 1)
      yield round(avg, 2)

def get_median(cnt_users):
  for i in range(len(cnt_users)):
    if (len(cnt_users[:i]) + 1) % 2 == 1:  # Если количество элементов нечетное
        median = sorted(cnt_users[:i+1])[(len(cnt_users[:i]) + 1) // 2]
    else:  # Если количество элементов четное
        middle1 = sorted(cnt_users[:i+1])[(len(cnt_users[:i]) + 1) // 2 - 1]
        middle2 = sorted(cnt_users[:i+1])[(len(cnt_users[:i]) + 1) // 2]
        median = (middle1 + middle2) / 2
    yield int(median)

def auto_renewal_sub(log_file_path):
  with open(log_file_path, 'r') as file:
      reader = file.readlines()

  #составляем список логов, содержащих слова "количество людей"
  #оставляем только количество
  cnt_users = [
      (
          row.split('|')[4].split(' ')[3].strip(','), int(row.split(' ')[-1])
          )
      for row in reader if 'количество людей' in row
      ]
  max_cnt_users = get_max(cnt_users)
  avg = list(get_avg(max_cnt_users))
  median = list(get_median(max_cnt_users))

  with open('auto_renewal_sub.txt', 'w') as file:
    file.write(
        'Среднее: {avg}\nМедиана: {median}'.format(
            avg=avg,
            median=median
            )
        )

auto_renewal_sub('auto_purchase.log')

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [ ]:
# Здесь будет скачиваться файл с эталонным ответом

!wget https://gist.github.com/Vs8th/846cbc97a722ab825cda4dda3b2b3434/raw/cor_auto_renewal.txt

import pandas as pd

user_answer = pd.read_csv('auto_renewal_sub.txt')
correct_answer = pd.read_csv('cor_auto_renewal.txt')

--2025-03-18 05:26:55--  https://gist.github.com/Vs8th/846cbc97a722ab825cda4dda3b2b3434/raw/cor_auto_renewal.txt
Resolving gist.github.com (gist.github.com)... 140.82.113.4
Connecting to gist.github.com (gist.github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/846cbc97a722ab825cda4dda3b2b3434/raw/cor_auto_renewal.txt [following]
--2025-03-18 05:26:56--  https://gist.githubusercontent.com/Vs8th/846cbc97a722ab825cda4dda3b2b3434/raw/cor_auto_renewal.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2431 (2.4K) [text/plain]
Saving to: ‘cor_auto_renewal.txt’

cor_auto_renewal.tx 100%[===================>]   2.37K  --.-KB/s    in 0s      

2025-03-18 05:2

In [ ]:
try:
  assert (user_answer == correct_answer).all().all(), 'Ответы не совпадают'
except Exception as err:
  raise AssertionError(f'При проверке возникла ошибка {repr(err)}')
else:
  print('Поздравляем, Вы справились и успешно прошли все проверки!!')

Поздравляем, Вы справились и успешно прошли все проверки!!


#### **Задача 3**

Напишите функцию `sub_renewal_by_day`, которая принимает на вход путь к файлу с логами и анализирует взаимосвязь дня продления подписки и количества продлений в этот день. Функция должна записать в файл `weekdays.txt` аналитическую записку в формате:

**`Количество обновлений подписки по дням недели:`**

**`Понедельник: 6`**

**`Вторник: 7`**

**`Среда: 8`**

**`...`**

**Решение**

Напишите свое решение ниже

In [3]:
from datetime import datetime
from collections import Counter

In [ ]:
def sub_renewal_by_day(file_path):
  """
  Создает файл в формате txt c подсчетами количества обовлений по дням недели.

    Args:
        file_path (str): имя файла с логами.
  """
  with open(file_path, 'r') as file:
    reader = file.readlines()

  #составляем список дней недели с успешными обновлениями подписки
  #(лог содержит слова "Обновляем подписку" и следующий лог не содержит error)
  #и подсчитываем количество записей по дням недели
  cnt_users = [
          datetime.strptime(
              row.split('|')[1].split(' ')[1], '%Y-%m-%d').isoweekday()
              for i, row in enumerate(reader)
              if (
                  i < len(reader) - 1
                  and 'Обновляем подписку' in row
                  and not reader[i+1].startswith('ERROR')
                  )
              or ('Обновляем подписку' in row and i >= len(reader) - 1)
          ]
  cnt_by_week = Counter(cnt_users)

  with open('weekdays.txt', 'w') as file:
    file.write(
        'Понедельник: {monday}\nВторник: {tuesday}\nСреда: {wednesday}\nЧетверг: {thursday}\nПятница: {friday}\nСуббота: {saturday}\nВоскресенье: {sunday}'.format(
            monday=cnt_by_week[1],
            tuesday=cnt_by_week[2],
            wednesday=cnt_by_week[3],
            thursday=cnt_by_week[4],
            friday=cnt_by_week[5],
            saturday=cnt_by_week[6],
            sunday=cnt_by_week[7]
            )
        )

sub_renewal_by_day('auto_purchase.log')

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [ ]:
# Здесь будет скачиваться файл с эталонным ответом

!wget https://gist.github.com/Vs8th/c4c382f50761b5b9e64795eb89d49fda/raw/cor_weekdays.txt

import pandas as pd

user_answer = pd.read_csv('weekdays.txt')
correct_answer = pd.read_csv('cor_weekdays.txt')

--2025-03-18 05:25:27--  https://gist.github.com/Vs8th/c4c382f50761b5b9e64795eb89d49fda/raw/cor_weekdays.txt
Resolving gist.github.com (gist.github.com)... 140.82.113.3
Connecting to gist.github.com (gist.github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/c4c382f50761b5b9e64795eb89d49fda/raw/cor_weekdays.txt [following]
--2025-03-18 05:25:27--  https://gist.githubusercontent.com/Vs8th/c4c382f50761b5b9e64795eb89d49fda/raw/cor_weekdays.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 238 [text/plain]
Saving to: ‘cor_weekdays.txt’

cor_weekdays.txt    100%[===================>]     238  --.-KB/s    in 0s      

2025-03-18 05:25:28 (7.20 MB/s) - ‘cor_

In [ ]:
try:
  assert (user_answer == correct_answer).all().all(), 'Ответы не совпадают'
except Exception as err:
  raise AssertionError(f'При проверке возникла ошибка {repr(err)}')
else:
  print('Поздравляем, Вы справились и успешно прошли все проверки!!')

Поздравляем, Вы справились и успешно прошли все проверки!!
